In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


In [2]:
import warnings
# Ignore all warnings
warnings.filterwarnings('ignore')

## 1. Load the Dataset

In [3]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')  # Assuming train.csv contains training data
test_data = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')    # Assuming test.csv contains test data


In [4]:
train_data.sample(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
111913,111913,Audi,S4 3.0 Prestige,2012,108000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,Black,None reported,Yes,20997
22339,22339,Audi,A4 2.0T Premium Plus,2015,96000,Gasoline,220.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,23000
137659,137659,Cadillac,XTS Luxury,2003,90000,Gasoline,275.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,A/T,Blue,Brown,None reported,Yes,37500
36375,36375,Nissan,Versa 1.8 S,2012,125500,Gasoline,122.0HP 1.8L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,9900
78111,78111,Ford,F-150 XLT,2021,26998,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Gray,Medium Light Camel,None reported,Yes,47950


In [5]:
test_data.sample(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
86822,275355,Kia,Sportage SX Turbo,2016,108559,Gasoline,2.4L I4 16V GDI DOHC,6-Speed Automatic,Hyper Red,Black,None reported,NaN
113455,301988,Chevrolet,Corvette Stingray w/3LT,2021,12000,Gasoline,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes
56678,245211,Ford,Explorer XLT,2022,8794,Gasoline,300.0HP 2.3L 4 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Black,None reported,Yes
86597,275130,Ford,F-150 Lariat,2015,159500,Gasoline,365.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,None reported,Yes
121547,310080,Porsche,718 Cayman GT4,2019,6581,Gasoline,414.0HP 4.0L Flat 6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes


## 2. Data Preprocessing

### Handling missing values

In [6]:
# Check for missing values
print(train_data.isnull().sum())

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64


In [7]:
# Check for percentage of  missing values
print(train_data.isnull().sum()*100/len(train_data))


id               0.000000
brand            0.000000
model            0.000000
model_year       0.000000
milage           0.000000
fuel_type        2.696080
engine           0.000000
transmission     0.000000
ext_col          0.000000
int_col          0.000000
accident         1.300568
clean_title     11.360876
price            0.000000
dtype: float64


In [8]:
# Impute missing values for numerical features
#train_data.fillna(train_data.median(), inplace=True)

# Impute missing values for categorical features
#train_data.fillna(train_data.mode().iloc[0], inplace=True)


The error occurs because some columns in dataset are categorical (e.g., car make, model, fuel type, color, etc.), and the machine learning algorithm using expects numeric input. To fix this,we'll need to convert the categorical data into a numeric format that the model can handle.

We can use either **Label Encoding** or **One-Hot Encoding** for this conversion depending on whether the categorical variable is *ordinal* or *nominal*.

Here’s how we can correct the code by applying the appropriate encoding to the categorical features:

**1. Label Encoding for Ordinal Categorical Variables**

*For features that have a meaningful order (like 'low', 'medium', 'high'), use LabelEncoder.*

**2. One-Hot Encoding for Nominal Categorical Variables**

*For features that don’t have any specific order (like car brand, fuel type, etc.), use OneHotEncoder.*

### Feature Encoding

`Convert categorical features into numeric using one-hot encoding or label encoding`

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Identify categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns
categorical_cols


Index(['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title'],
      dtype='object')

In [10]:
# Apply OneHotEncoding for nominal categorical variables (e.g., car make, fuel type, etc.)
train_data = pd.get_dummies(train_data, columns=categorical_cols, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

**pd.get_dummies():** This function converts categorical columns into multiple binary (0/1) columns (one-hot encoding), which allows the model to interpret them.

**drop_first=True:** Drops the first category to avoid multicollinearity.

**Reindexing Test Data:** After applying one-hot encoding, the columns generated in the training and test sets may differ. Using reindex ensures that the test set has the same columns as the training set, with missing columns filled with zeros.

**Handling Target Column in Test Data:** Since the test set does not have the price column, we remove it from the test set to avoid errors during prediction.

In [11]:
train_data.sample(5)

,id,model_year,milage,price,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,...,int_col_Tupelo,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_None reported
14185,14185,2018,58000,59000,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
79146,79146,2016,78110,28000,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
167041,167041,2023,3000,94000,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
109575,109575,2022,16184,18688,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
135968,135968,2022,32437,75500,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [12]:
test_data.sample(5)

,id,model_year,milage,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,brand_Buick,...,int_col_Tupelo,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_None reported
107176,295709,2021,13800,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
43753,232286,2019,111000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21984,210517,2017,15600,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
42100,230633,2023,8200,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
11343,199876,1998,98000,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Align columns of test data with training data (since one-hot encoding can produce different sets of columns)
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)
test_data.sample(5)

,id,model_year,milage,price,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,...,int_col_Tupelo,int_col_Very Light Cashmere,int_col_WHITE,int_col_Walnut,int_col_Whisper Beige,int_col_White,int_col_White / Brown,int_col_Yellow,int_col_–,accident_None reported
30584,219117,2016,72500,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
86644,275177,2016,84352,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3695,192228,2008,127500,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
55349,243882,2009,162950,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
51667,240200,2013,120000,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
# Drop 'price' column from test data as it's not available in test
test_data.drop(columns=['price'], errors='ignore', inplace=True)

In [15]:
# Check to ensure the data is now numeric
print(train_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Columns: 3603 entries, id to accident_None reported
dtypes: bool(3599), int64(4)
memory usage: 652.9 MB
None


In [16]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Columns: 3602 entries, id to accident_None reported
dtypes: bool(3587), int64(15)
memory usage: 444.3 MB
None


## 3. Feature Engineering

`Create additional features (like car age, mileage categories, etc.) to improve model performance`

In [17]:
# Example: Create a new feature for car age
train_data['car_age'] = 2024 - train_data['model_year'] 
test_data['car_age'] = 2024 - test_data['model_year']


In [18]:
print(train_data['car_age'].head())

0    17
1    22
2    22
3     7
4     3
Name: car_age, dtype: int64


In [19]:
# Drop unnecessary columns like 'year'
train_data.drop('model_year', axis=1, inplace=True)
test_data.drop('model_year', axis=1, inplace=True)

## 4. Splitting the Data

*Split the data into features (**X**) and target (**y**)*

In [20]:
X = train_data.drop(columns=['price', 'id'])  # Drop target and id from features
y = train_data['price']

In [21]:
# Prepare the test features (without price column)
X_test = test_data.drop(columns=['id'])

## 5. Train-Test Split

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## 6. Model Training

- We will use Gradient Boosting or Random Forest, which are typically effective for tabular data. 
- Let’s use XGBoost for this purpose as it is fast and efficient.



In [23]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Initialize XGBoost regressor
xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8, random_state=42)

In [24]:
# Fit the model with early stopping
xgb_model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_val, y_val)],  # Validation set for early stopping
    early_stopping_rounds=50,   # Stop if performance doesn't improve after 50 rounds
    verbose=False               # Suppress training output
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [25]:
# Make predictions on the validation set
y_val_pred = xgb_model.predict(X_val)

In [26]:
# Calculate RMSE
rmse = mean_squared_error(y_val, y_val_pred, squared=False)
print(f'Validation RMSE: {rmse}')

Validation RMSE: 69294.54372656536


## 7. Make Predictions on Test Set

In [27]:
# Make predictions on the test set
test_predictions = xgb_model.predict(X_test)

In [28]:
test_predictions

array([21629.504, 71963.125, 57056.19 , ..., 23772.69 , 14266.046,
       36072.9  ], dtype=float32)

In [29]:
# Prepare the submission file
submission = pd.DataFrame({'id': test_data['id'], 'price': test_predictions})

In [30]:
submission

,id,price
0,188533,21629.503906
1,188534,71963.125000
2,188535,57056.191406
3,188536,27783.203125
4,188537,36993.617188
...,...,...
125685,314218,24584.654297
125686,314219,50949.507812
125687,314220,23772.689453
125688,314221,14266.045898


In [31]:
submission.to_csv('submission.csv', index=False)